# Setup

In [ ]:
%pip install requests requests-oauthlib gitpython

In [ ]:
import os, sys, json, tempfile, shutil, time, webbrowser
from pathlib import Path
from urllib.parse import urlparse, parse_qs
from git import Repo
from datetime import datetime
import requests
from requests_oauthlib import OAuth1Session

# Notebook-friendly: no Rich, no inquirer
ZOTERO_CLIENT_KEY    = "a794e237f0ed3ee439b3"
ZOTERO_CLIENT_SECRET = "454dee43d7e3b8240f62"
REQUEST_TOKEN_URL    = "https://www.zotero.org/oauth/request"
AUTHORIZE_URL        = "https://www.zotero.org/oauth/authorize"
ACCESS_TOKEN_URL     = "https://www.zotero.org/oauth/access"
ZOTERO_PAGE_LIMIT    = 100
CONFIG_PATH          = Path.home()/".config"/"zotero_overleaf"/"config.json"

In [ ]:
def load_configs(path):
    if path.exists():
        with open(path,"r") as f:
            return json.load(f)
    return {"zotero_credentials": [], "overleaf_tokens": [], "overleaf_projects": []}

def save_configs(path, cfg):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path,"w") as f:
        json.dump(cfg, f, indent=2)

def get_zotero_oauth_token_oob():
    oauth = OAuth1Session(
        ZOTERO_CLIENT_KEY,
        client_secret=ZOTERO_CLIENT_SECRET,
        callback_uri="oob"
    )
    # 1) fetch request token
    fetch = oauth.fetch_request_token(REQUEST_TOKEN_URL)
    token, secret = fetch["oauth_token"], fetch["oauth_token_secret"]
    # 2) get auth URL & prompt user
    auth_url = oauth.authorization_url(AUTHORIZE_URL)
    print("Visit this URL, log in, authorize, and copy the verifier code:")
    print(auth_url)
    verifier = input("Verifier code: ").strip()
    # 3) fetch access token
    oauth = OAuth1Session(
        ZOTERO_CLIENT_KEY,
        client_secret=ZOTERO_CLIENT_SECRET,
        resource_owner_key=token,
        resource_owner_secret=secret,
        verifier=verifier
    )
    tokens = oauth.fetch_access_token(ACCESS_TOKEN_URL)
    return {
        "user_id":   tokens.get("userID") or tokens.get("user_id"),
        "api_key":   tokens["oauth_token"],
        "api_secret":tokens["oauth_token_secret"],
        "user_name": tokens.get("username","")
    }

def parse_zotero_url(url):
    """Parse a Zotero library/collection URL into group flag, user/group ID, and optional collection ID."""
    parts = urlparse(url).path.strip('/').split('/')
    if parts[0] == 'groups':
        is_group = True
        user_id = parts[1]
        collection = parts[3] if len(parts) >= 4 and parts[2] == 'collections' else None
    else:
        is_group = False
        user_id = parts[0]
        collection = parts[2] if len(parts) >= 3 and parts[1] == 'collections' else None
    return is_group, user_id, collection

def _get_all_subcollections(cred, is_group, user_id, parent_id):
    base = 'groups' if is_group else 'users'
    url = f"https://api.zotero.org/{base}/{user_id}/collections/{parent_id}/collections"
    params = {'key': cred['api_key'], 'limit': ZOTERO_PAGE_LIMIT}
    sub_ids, next_url = [], url
    while next_url:
        resp = requests.get(next_url, params=params)
        resp.raise_for_status()
        data = resp.json()
        for c in data:
            sub_ids.append(c['key'])
        link = resp.headers.get('Link','')
        next_url = None
        for part in requests.utils.parse_header_links(link.rstrip('>')):
            if part.get('rel') == 'next': next_url = part['url']; break
    all_desc = []
    for cid in sub_ids:
        all_desc.append(cid)
        all_desc.extend(_get_all_subcollections(cred, is_group, user_id, cid))
    return all_desc

def fetch_zotero_bib(cred, is_group, user_id, coll_id=None):
    base = 'groups' if is_group else 'users'
    # gather collections
    if coll_id:
        coll_ids = [coll_id] + _get_all_subcollections(cred, is_group, user_id, coll_id)
    else:
        coll_ids = [None]
    parts = [f"% Zotero Sync @ {datetime.now().isoformat()}\n"]
    for coll in coll_ids:
        url = (f"https://api.zotero.org/{base}/{user_id}/collections/{coll}/items/top" if coll
               else f"https://api.zotero.org/{base}/{user_id}/items/top")
        params = {'format':'bibtex','key':cred['api_key'],'limit':ZOTERO_PAGE_LIMIT}
        resp = requests.get(url, params=params)
        resp.raise_for_status()
        parts.append(resp.text)
    return "\n".join(parts)

def clone_or_update_repo(git_url, token, local_dir=None):
    if local_dir:
        path = Path(local_dir).expanduser()
    else:
        path = Path(tempfile.mkdtemp())
    parsed = urlparse(git_url)
    domain = parsed.netloc.split("@")[-1]
    domain_and_path = domain + parsed.path
    auth = f"{parsed.scheme}://git:{token}@{domain_and_path}"
    if (path/'.git').exists():
        repo = Repo(path); repo.remotes.origin.pull()
    else:
        repo = Repo.clone_from(auth, path)
    return repo, path

def update_bib_and_push(repo, path, bib_str):
    bibfile = path/'references.bib'
    bibfile.write_text(bib_str, encoding='utf-8')
    repo.git.add('references.bib')
    if repo.is_dirty():
        repo.index.commit('Sync Zotero .bib')
        repo.remotes.origin.push()
        print('✅ Pushed updates')
    else:
        print('ℹ️ No changes')

In [ ]:
def main():
    configs = load_configs(CONFIG_PATH)
    cred = get_zotero_oauth_token_oob()
    configs.setdefault('zotero_credentials', []).append(cred)
    save_configs(CONFIG_PATH, configs)

    zotero_url = input(
        "Enter Zotero library/collection URL (e.g. https://www.zotero.org/users/123456/collections/ABCDEF): "
    ).strip()
    is_group, zot_user_id, zot_collection = parse_zotero_url(zotero_url)

    overleaf_token = input("Overleaf Git token: ").strip()
    git_url        = input("Overleaf Git clone URL: ").strip()
    local_dir      = None  # or specify a path

    repo, path = clone_or_update_repo(git_url, overleaf_token, local_dir)
    bib = fetch_zotero_bib(cred, is_group, zot_user_id, zot_collection)
    update_bib_and_push(repo, path, bib)

    # Cleanup if temp
    if local_dir is None:
        try:
            shutil.rmtree(path)
        except Exception as e:
            print(f"⚠️ Failed to clean up temporary directory: {e}")
    print("Done! Your Zotero library has been synced to Overleaf.")


# Execute

In [ ]:
main()